<a href="https://colab.research.google.com/github/angeruzzi/MachineLearningRef/blob/main/scanModels_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Comparação de Modelos


Autor: [Alessandro S. Angeruzzi](https://www.linkedin.com/in/alessandroangeruzzi/)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split #para aplicação do Holdout
from sklearn.metrics import mean_squared_error #EQM
from sklearn.metrics import mean_absolute_error #EMA

from sklearn.linear_model import LinearRegression #Regressão Linear
from sklearn.neighbors import KNeighborsRegressor #KNN
from sklearn.tree import DecisionTreeRegressor #Arvore de Decisão

In [ ]:
def HoldoutRepetido(modelo, X, y, R = 100, tsize = 0.25):
  eqms = []
  emas = []

  for i in range(R):
    Xtreino, Xteste, ytreino, yteste = train_test_split(X, y, test_size = tsize)
    mdl = modelo
    mdl.fit(Xtreino, ytreino)
    preditos = mdl.predict(Xteste)
    eqm = mean_squared_error(yteste, preditos)
    ema = mean_absolute_error(yteste, preditos) #np.mean(abs(yteste - preditos))
    eqms.append(eqm)
    emas.append(ema)
  
  res1 = np.array(eqms).mean().round(2)
  res2 = np.array(emas).mean().round(2)
  return res1, res2

In [ ]:
def ScanModel(modelos, nomes, X, y, R = 100, tsize=0.25):
  res1 = []
  res2 = []  
  for modelo in modelos:
    ret = HoldoutRepetido(modelo, X, y, R, tsize)
    res1.append(ret[0])
    res2.append(ret[1])

  resultados = pd.DataFrame({'EQM':res1, 'EMA':res2}, index=nomes)
  return resultados

In [ ]:
#Dados Utilizados
fonte = 'https://raw.githubusercontent.com/angeruzzi/Datasource/main/exampleDataInsurance.csv'
dados = pd.read_csv(fonte)
#Tratativas
dados['smoker'] = dados['smoker'].map({'yes':1, 'no':0})
dados['charges'] = dados['charges']/1000
dados = dados[['bmi', 'smoker', 'charges']]

X = dados[['bmi', 'smoker']]
y = dados['charges'] 

In [ ]:
modelos = [LinearRegression(), 
           KNeighborsRegressor(n_neighbors = 5), 
           KNeighborsRegressor(n_neighbors = 15),
           KNeighborsRegressor(n_neighbors = 25),
           KNeighborsRegressor(n_neighbors = 35),
           KNeighborsRegressor(n_neighbors = 45),
           DecisionTreeRegressor(max_depth = 1),
           DecisionTreeRegressor(max_depth = 2),
           DecisionTreeRegressor(max_depth = 3),
           DecisionTreeRegressor(max_depth = 4),
           DecisionTreeRegressor(max_depth = 5)]

nomes = ['Regressão Linear',
         'KNN - K = 5', 
         'KNN - K = 15', 
         'KNN - K = 25', 
         'KNN - K = 35', 
         'KNN - K = 45',
         'Árvore - Prof = 1',
         'Árvore - Prof = 2',
         'Árvore - Prof = 3',
         'Árvore - Prof = 4',
         'Árvore - Prof = 5']

In [ ]:
resultados = ScanModel(modelos, nomes, X, y, R = 1000, tsize=0.25)
resultados.sort_values(by='EQM')

,EQM,EMA
Árvore - Prof = 3,34.85,4.42
Árvore - Prof = 2,35.00,4.44
Árvore - Prof = 4,35.91,4.48
Árvore - Prof = 5,37.39,4.55
KNN - K = 5,47.16,4.99
Regressão Linear,50.64,5.43
KNN - K = 15,51.13,5.11
Árvore - Prof = 1,55.93,5.67
KNN - K = 25,59.67,5.46
KNN - K = 35,73.03,6.11
